# Establishment probabilities

This notebook can be used to 
1. create input files for simulations to compute establishment probabilities
2. analyze simulation data and create result plots

## Import libraries and set global settings

In [ ]:
import numpy as np
import plot
import module
import pickle,math
import matplotlib
import matplotlib.pyplot as plt
import itertools
import re
import pandas as pd
import json
import scipy
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import string
from ipywidgets import IntProgress
from IPython.display import display


In [ ]:
font = {'family' : 'Arial','size'   : 6}
matplotlib.rc('font', **font)

## Simulation


Set simulation parameters and create simulation input files

In [ ]:
# excecution time: ca 6 min (stores 76k files on hard disk)
b_list=np.array([  0.01, 0.001 ])
f_list=np.array([  10**(-7) ])
n_list=np.array(range(1,20+1))
s_list=np.linspace(0.05,0.50,19)
seed_list=np.array(range(1000))

def parset(par):
    b,f,n,s,seed=par
    return {"b":float(b),"f":float(f),"n":int(n),"s":float(s),"seed":int(seed),
            "file":"establishment_output/"+re.sub('[^a-zA-Z0-9_]','_',
                                '_'.join(map(str,
                                ("b",b,"f",f,"s", "%.3f" % s,"n",n,"seed",seed)))
        )}

l=list(itertools.product(b_list,f_list,n_list, s_list, seed_list))
parsets=list(map(parset,l))
print(len(parsets),' parameter combinations defined')

print('example parameter set:', parsets[0])

# write parametersets to input files
f = IntProgress(min=0, max=len(parsets)/100) # instantiate the bar
display(f);

for i in range(len(parsets)):
    with open( "establishment_input/inputfile_"+str(i), 'w') as file:
        file.write(json.dumps( parsets[i]))
    if not i%100:   
        f.value+=1
    
print("Input files saved. ")



Run simulations. Needs a running ipp cluster.

In [ ]:
if do_sim:

    # initialize ipp cluster for simulations
    try:
        with open('profile.temp') as f:
            profile = f.readlines()
        profile=profile[0].rstrip("\n")

    except:
        print("No profile file found. Using standard profile.")
        profile=None
    r=ipp.Client(profile=profile)

    %px import src.module
    lv=r.load_balanced_view()

    r[:].push(dict(s_list=s_list))
    # run simulation for D days

    lv.map_sync(lambda p: src.module.stochbottleSim(
        n=p['n'],f=p['f'],rwt=1.-p['s'],b=p['b'],seed=p['seed'],file=p['file'],establishment=True)
        if p['seed']>=100
        else None
    ,
    parsets[200:201]
    # ptest
    )


## Results

In [ ]:
# # uncomment for analyzing simulation data from output files

# est_array=np.zeros((len(b_list),len(f_list),len(n_list),len(s_list),len(seed_list)))
# est_array[:]=np.nan
# fnf_counter=0
# for p in parsets:
#     if p['seed']==0:
#         print('%i %.3f' % (p['n'],p['s']),end=', ')
#     try:
#         sim=pickle.load(open(p['file'],'rb'))
        
#         est=sim[1][-1][-1]!=0   # check if, at end of days [0], 
#                                 # at the last day [-1] 
#                                 # the homozygotes [-1] are still present
#                                 # (establishment)
#         est_array[tuple(whereisp(p))]=est
#     except:
#         print('no file found ', p['file'])
#         fnf_counter+=1
# print(fnf_counter,' files not found')

# pickle.dump(est_array,open('establishment_output/est.dat','wb'))

est_array=pickle.load(open('establishment_output/est.dat','rb'))

In [ ]:
est_array1=np.sum(est_array,axis=-1)/np.shape(est_array)[-1]

fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(16/2.54,6.5/2.54))
fig.subplots_adjust(left=0.15, bottom=None, right=None, top=None, wspace=0.1, hspace=None)


for bi, b in enumerate(b_list):
    Z=np.flip(est_array1[bi,0].T,axis=0)[:,:]
    ax1=ax[bi].imshow(Z, vmax=1.,vmin=0.,cmap=plt.get_cmap('plasma'))
    N,S=np.meshgrid(n_list,s_list)
    ax[bi].contour(range(len(n_list)),range(len(s_list)),Z,levels=[0.50,1.0],linestyles=['dashed','dashed'],linewidths=[1.,1.])
    ax[bi].set_title('$b=%g$' % b)
    xticks=np.array([1,5,10,15,20])
    ax[bi].set_xticks(range(len(n_list)), minor=True)
    ax[bi].set_xticks( xticks-1, minor=False)
    ax[bi].set_xticklabels(['%i' % n for n in  xticks])
    yticks=np.array([0.05,0.10,0.20,0.30,0.40,0.50])
    # ax[bi].set_yticks(range(len(s_list)), minor=True)
    ytickspositions=np.array([np.argwhere(yt==s_list)[0] for yt in yticks ])[:,0]
    ax[bi].set_yticks( -ytickspositions+len(s_list)-1, minor=False)
    ax[bi].set_xlabel('Replicon copy number $n_\mathrm{rc}$')
    if bi==0:
        ax[bi].set_yticklabels(['%.2f' % s for s in  yticks])
        ax[bi].set_ylabel('Strength of selection $s$')
        
    else:
        ax[bi].set_yticklabels([])
        ax[bi].tick_params(length=0,axis='y')

    # ax[bi].set_yticks(range(len(s_list)))
    # ax[bi].set_yticklabels(['%.2f'%s for s in np.flip(s_list)],size=8)
    ax[bi].grid(linewidth=.25)
    # In [11]: grid(b=True, which='major', color='b', linestyle='-')
    ax[bi].text(-0.05, 1.05, string.ascii_uppercase[bi], transform=ax[bi].transAxes, 
        size=8, weight='bold')


fig.subplots_adjust(right=.79)
cbar_ax = fig.add_axes([.82, ax[0].get_position().y0, 0.025, ax[0].get_position().y1-ax[0].get_position().y0])
cb=fig.colorbar(ax1, cax=cbar_ax,fraction=0.02)
cb.set_label(label='Establishment probability $p_\mathrm{est}$', rotation=270, labelpad=10)

fig.savefig('../figures-plots/final-figures-main/fig-4.pdf')